In [ ]:
import chromadb
import pandas as pd
from chromadb.utils import embedding_functions
import pythainlp
import pythainlp.util
from pythainlp import word_tokenize


In [3]:
data = pd.read_csv('jobsdb_20240829.csv')
data["Description"] = data["Description"].fillna(data["Job"])

print(data["Description"])

0       responsibilities:\nสร้างยอดขายโดยกําหนดกลยุทธ์...
1       job responsibilities:\ncontact and coordinate,...
2       key responsibilities\nyou will develop new and...
3       duties & responsibilities\nthis position is re...
4       responsibilities:\n-handle company’s overall p...
                              ...                        
1629                   Project Engineer - Decommissioning
1630    job description\nfind new customers (sales hun...
1631    job description\n*experience in the jewellery ...
1632    responsibility\nexamine and analyze internal i...
1633    core responsibilities:\nplant operations and f...
Name: Description, Length: 1634, dtype: object


In [4]:
job_ids = data.index.to_list()
job_names = data['Job'].tolist()
documents = data['Description'].str.replace('\n', '').tolist()

In [5]:
embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction("paraphrase-multilingual-mpnet-base-v2", normalize_embeddings=True)

client = chromadb.PersistentClient(path="Database")
COLLECTION_NAME = "job_docs"

c:\Users\krasi\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\krasi\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
collection = client.get_or_create_collection(
    name = COLLECTION_NAME,
    embedding_function = embedding_func,
    metadata = {"hnsw:space": "cosine"},)

In [8]:
print("documents:", documents)
print("job_ids:", job_ids)
print("job_names:", job_names)
print("data:", data)

documents: ['responsibilities:สร้างยอดขายโดยกําหนดกลยุทธ์และวิเคราะห์รูปแบบบริการต่างๆคำนวนโครงสร้างต้นทุนรักษาความสัมพันธ์กับลูกค้าจัดหาและเสนอราคาแก่ลูกค้าปฏิบัติตามข้อกาหนด iso และงานอื่นๆที่ได้รับมอบหมาย', 'job responsibilities:contact and coordinate, receive information from customers in cad, dwg to make quotation submit to customer.control the profit level of the items being produced.internal coordination activities that customers come in, such as quality , delivery, va/ve.organize a kick-off apqp team meeting to jointly examine and review capacity in creating tools with relevant parties.design change follow up.price adjustment.kick off all activity.build relation with customer.', 'key responsibilitiesyou will develop new and improve existing manufacturing methods to increase efficiency and improve quality.partner with technical team to ensure existing formulation are optimized to increase efficiency and reduce operation cost.you will lead root cause investigations for manufactur

In [7]:
collection.add(
    documents=documents,
    ids=[str(i) for i in job_ids],
    metadatas=[{"name": str(job_names[n])} for n in range(len(data))]
)

In [14]:
pythainlp.sent_tokenize(data['Description'].tolist()[2], engine='crfcut')

['key responsibilities\nyou will develop new and improve existing manufacturing methods to increase efficiency and improve quality.',
 '\npartner with technical team to ensure existing formulation are optimized to increase efficiency and reduce operation cost.',
 '\nyou will lead root cause investigations for manufacturing related issues both internally and in the field\noversee processes in detail and prepare the implementation plans to ensure the qualities are met\nyou will ensure consistent operating practices in production\nyou will implement the transition of new technology to make sure production continues without any hassles\nimprove the present manufacturing or production processes to boost productivity, quality, and profitability for a company\nresolve day- to- day technical problem on the production line and ensure no issues from the designed production processes.',
 '\nunderstand and utilize fmea’s as part of product design, process design and operations to drive standardiza

In [9]:
collection.query(query_texts="ชอบเล่นเกม")

{'ids': [['163',
   '140',
   '562',
   '527',
   '659',
   '370',
   '1074',
   '1459',
   '808',
   '1475']],
 'distances': [[0.6917482614517212,
   0.7623139023780823,
   0.7712395787239075,
   0.8231058120727539,
   0.8231058120727539,
   0.824992835521698,
   0.8346697092056274,
   0.834669828414917,
   0.8377996683120728,
   0.842499852180481]],
 'metadatas': [[{'name': 'พนักงานแอดมิน'},
   {'name': 'Content Creator'},
   {'name': 'Assistant Controller'},
   {'name': 'Content Creator (Marketing)'},
   {'name': 'Content Creator (Marketing)'},
   {'name': 'ผู้จัดการเขต (ร้านปันกัน)'},
   {'name': 'อาจารย์ คณะนิเทศศาสตร์สาขาการกระจายเสียงแพร่ภาพและสตรีมมิ่ง'},
   {'name': 'อาจารย์ คณะนิเทศศาสตร์สาขาการกระจายเสียงแพร่ภาพและสตรีมมิ่ง'},
   {'name': 'Technologist'},
   {'name': 'ผู้ช่วยผู้จัดการฝ่ายบัญชี'}]],
 'embeddings': None,
 'documents': [['พนักงานแอดมิน',
   'หน้าที่และความรับผิดชอบ- สร้างสรรค์คอนเทนต์รูปแบบใหม่ๆ รูปภาพ กราฟิก คลิปวิดีโอ หรืออื่นๆ ให้เหมาะกับแพลตฟอร์มและกลุ่มเป้

In [8]:
def retrieve_relevant_jobs(query, top_k=5):
    results = collection.query(query_texts=query, n_results=top_k)

    if results and "metadatas" in results:
        retrieved_jobs = []
        for i in range(len(results["ids"][0])):
            job_info = {
                "id": results["ids"][0][i],
                "name": results["metadatas"][0][i].get("name", ""),
                "description": results["documents"][0][i]
            }
            retrieved_jobs.append(job_info)
        return retrieved_jobs
    return []


In [48]:
query = "คอมพิวเตอร์"
retrieved_jobs = retrieve_relevant_jobs(query)

print("งานที่เกี่ยวข้อง:")
for job in retrieved_jobs:
    print(f"- ID: {job['id']}")
    print(f"  ชื่องาน: {job['name']}")
    print(f"  รายละเอียด: {job['description']}")

งานที่เกี่ยวข้อง:
- ID: 808
  ชื่องาน: Technologist
  รายละเอียด: Technologist
- ID: 637
  ชื่องาน: IT Technician
  รายละเอียด: IT Technician
- ID: 190
  ชื่องาน: IT Support
  รายละเอียด: รายละเอียดงานสนับสนุนงานด้านระบบคอมพิวเตอร์ให้กับองค์กร โดยมีหน้าที่ดูแลและแก้ไขปัญหา hardware, software เช่น word, excel, sapb1, expres, network, internetจัดซื้อ hardware, softwareตรวจเช็คและซ่อมบำรุง hardware, software, network & internet
- ID: 104
  ชื่องาน: IT Support
  รายละเอียด: tasks & responsibilities1.ดูแลและแก้ปัญหาด้านคอมพิวเตอร์ในองค์กร เช่น microsoft outlook, microsoft office, window และอื่นๆ2. ตรวจสอบและบำรุงรักษาฮาร์ดแวร์และระบบไอทีทั้งหมด เช่น access point, cctv, file server, internet, firewall3. ติดตั้งอุปกรณ์ฮาร์ดแวร์ เช่น lan, cctv, เครื่องพิมพ์, โทรศัพท์, เครื่องแฟกซ์, router เป็นต้น4. ตรวจสอบความเร็วความพร้อมใช้งานของ internet และแก้ไขปัญหาหากเกิดปัญหา5. ดูแลทุกเครือข่ายของบริษัทให้ปลอดภัยจาก virus6.ดูแลจัดเตรียมห้องประชุม อุปกรณ์ต่างๆในห้องประชุม โปรเจ็กเตอร์ ไมค์ ลำโพง กล้องให้

In [11]:
def retrieve_relevant_docs(query, top_k=5):
    results = collection.query(query_texts=query, n_results=top_k)
    return results["metadatas"][0] if results else []

query = "ออกแบบโปรแกรม"
retrieved_docs = retrieve_relevant_docs(query)

for doc in retrieved_docs:
    print(f"- {doc}")

- {'name': 'System Analyst'}
- {'name': 'Die Design Engineer'}
- {'name': 'IT Developer'}
- {'name': 'Technologist'}
- {'name': 'Back-End Developer (NodeJS)'}
